未完成

url: https://www.youtube.com/watch?v=cU2RmYaR44I&t=1s

In [ ]:
# !apt-get remove mecab libmecab-dev mecab-ipadic-utf8
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3

In [ ]:
# https://github.com/SamuraiT/mecab-python3#common-issues
!pip install unidic-lite

In [ ]:
# 形態素解析
import MeCab

m = MeCab.Tagger()

def mecab_sep(text):
    node = m.parseToNode(text)

    words_list = []

    while node:
        if node.feature.split(",")[0] == "名詞":
            words_list.append(node.surface)
        elif node.feature.split(",")[0] == "動詞":
            words_list.append(node.feature.split(",")[6])
        elif node.feature.split(",")[0] == "形容詞":
            words_list.append(node.feature.split(",")[6])

        node = node.next

    return words_list[1:-1]

In [ ]:
# ベクトル化
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec

### Bag-of-words / tf-idf
def calc_vecs(docs):
    vectorizer = TfidfVectorizer(analyzer=mecab_sep)
    vecs = vectorizer.fit_transform(docs)
    return vecs.toarray()

### Doc2Vec
model = Doc2Vec.load("jawiki.doc2vec.dbow300d/jawiki.doc2vec.dbow300d.model")
def calc_vecs_d2v(docs):
    vecs = []
    for d in docs:
        vecs.append(model.infer_vector(mecab_sep(d)))
    return vecs


In [ ]:
#サンプルテキストの読み込み
import glob
docs = []

for x in glob.glob("text/*/*.txt"):
    text = ""
    with open(x) as f:
        next(f)
        next(f)
        for line in f:
            text = text + line
    f.close()
    docs.append((x.split("/")[1],text))

In [ ]:
import pandas as pd
df_docs = pd.DataFrame(docs,columns=["カテゴリ","内容"])

In [ ]:
all_docs_vecs = calc_vecs_d2v(pd_docs["内容"])

In [ ]:
from sklearn.cluster import KMeans

vec = KMeans(n_clusters=9)
vec.fit(all_docs_vecs)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
vec.labels_

array([6, 6, 6, ..., 1, 1, 1], dtype=int32)

In [ ]:
df_docs["クラス"] = vec.labels_

In [ ]:
df_docs

,カテゴリ,内容,クラス
0,movie-enter,【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か\n 2005年11月...,6
1,movie-enter,藤原竜也、中学生とともにロケット打ち上げに成功\n 「アンテナを張りながら生活をしていけばい...,6
2,movie-enter,『戦火の馬』ロイヤル・プレミアにウィリアム王子＆キャサリン妃が出席\n 3月2日より全国ロー...,6
3,movie-enter,香里奈、女子高生100人のガチンコ質問に回答「ラーメンも食べる」\n 女優の香里奈が18日、...,6
4,movie-enter,ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」\n 5日、東京・千代田区の...,6
...,...,...,...
7371,smax,好きな戦士を作ってドラゴンボールの世界で天下一武道会優勝だ！「挑戦！天下一武道会」【Andr...,2
7372,smax,NTTドコモ、GALAXY SIII SC-06DとF-09D ANTEPRIMAの発売日を...,1
7373,smax,NTTドコモ、Android向け「docomo Wi-Fiかんたん接続アプリ」をバージョンア...,1
7374,smax,NTTドコモ、PRADA Phone by LG L-02Dのデコメ絵文字popが正常に表示...,1


In [ ]:
pd.crosstab(df_docs["カテゴリ"],df_docs["クラス"])

クラス,0,1,2,3,4,5,6,7,8
カテゴリ,,,,,,,,,
dokujo-tsushin,8,0,0,3,9,86,31,1,733
it-life-hack,108,6,162,13,114,18,24,419,7
kaden-channel,72,2,4,2,704,20,6,49,6
livedoor-homme,8,1,24,13,37,186,21,29,193
movie-enter,11,0,1,5,6,6,826,1,15
peachy,9,0,7,9,21,569,87,8,133
smax,1,645,198,0,15,2,0,10,0
sports-watch,106,0,0,791,2,1,0,0,1
topic-news,709,0,1,43,9,1,2,0,6
